In [1]:
import os

os.chdir("C:/Users/anton/Desktop/University/AI_Lab/Git_repo")

In [20]:
import pandas as pd
import torch
import xgboost as xgb
from fix_data import add_label_T
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score
from torch import nn
from torch import optim
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.svm import SVC

In [3]:
df = pd.read_csv("data/SmartSeq/HCC1806_SmartS_Filtered_Normalised_3000_Data_train.txt", sep=" ")
df = add_label_T(df, dropname=True)
df

,DDIT4,ANGPTL4,CALML5,KRT14,CCNB1,IGFBP3,AKR1C2,KRT6A,NDRG1,KRT4,...,ZYG11A,NRG1,RBMS3,VCPIP1,LINC02693,OR8B9P,NEAT1,ZDHHC23,ODAD2,label
0,0,48,0,321,298,82,6250,634,0,0,...,10,136,0,0,29,0,29,0,0,1
1,8739,2101,55,96,1824,1938,62,0,522,413,...,0,264,0,134,68,0,213,0,0,0
2,13098,14032,0,0,1616,247,430,907,348,0,...,0,38,0,0,0,0,92,0,0,0
3,2880,356,0,6211,3,3430,79,1953,592,176,...,0,16,0,4,1,0,1,0,0,0
4,7777,5661,4383,0,145,4618,246,85,206,0,...,1,25,0,0,0,0,128,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,79,5,0,2098,554,99,798,4211,64,0,...,0,264,0,10,34,0,24,0,0,1
178,167,1,0,27,3585,134,3,596,11,0,...,0,120,25,0,53,0,177,53,0,1
179,10186,3147,0,149,1012,111,716,617,744,0,...,0,55,0,157,43,0,113,0,0,0
180,238,41,2867,814,2207,297,1023,2879,1,0,...,0,42,0,26,0,0,13,5,0,1


In [60]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("label", axis=1), df["label"], test_size=0.2)
X, y = df.drop("label", axis=1), df["label"]

In [5]:
param_grid = {
    "penalty": ["l1", "l2"],
    "C": [10**x for x in range(-3, 3)]
}
grid_search = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_, grid_search.best_score_)

{'C': 1, 'penalty': 'l1'} 1.0


In [68]:
cross_val_score(LogisticRegression(penalty="l1", C=1.2, solver="liblinear", random_state=42), X, y, cv=20)

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.88888889, 1.        , 1.        , 1.        ])

In [61]:
log_reg = LogisticRegression(penalty="l1", C=0.1, solver="liblinear")
log_reg.fit(X_train, y_train)
print(log_reg.score(X_test, y_test))
print(log_reg.score(X_train, y_train))
coef = log_reg.coef_.tolist()[0]
n = 0
for c in coef:
    if abs(c) > 0:
        n += 1
print(n)

1.0
1.0
53


In [8]:
param_grid = {
    "alpha": [10**x for x in range(-5, 5)]
}
grid_search_xgb = GridSearchCV(xgb.XGBClassifier(), param_grid)
grid_search_xgb.fit(X_train, y_train)
print(grid_search_xgb.best_score_, grid_search_xgb.best_params_)

KeyboardInterrupt: 

In [11]:
X_pred = pd.read_csv("data/SmartSeq/HCC1806_SmartS_Filtered_Normalised_3000_Data_test_anonim.txt", sep=" ").T
X_pred

,DDIT4,ANGPTL4,CALML5,KRT14,CCNB1,IGFBP3,AKR1C2,KRT6A,NDRG1,KRT4,...,MST1R,ZYG11A,NRG1,RBMS3,VCPIP1,LINC02693,OR8B9P,NEAT1,ZDHHC23,ODAD2
1,0,0,0,169,233,227,521,577,0,0,...,0,0,176,0,0,48,0,118,6,0
2,2475,0,0,0,3537,843,214,0,3,0,...,106,0,333,0,18,65,0,39,0,0
3,9088,2143,0,0,124,0,0,0,415,0,...,299,0,0,45,0,0,0,172,0,0
4,6909,3086,0,909,78,466,508,193,453,0,...,319,0,95,167,0,5,0,31,0,10
5,13655,2196,0,1,1,14688,3872,184,884,176,...,0,0,1,0,0,0,0,224,0,0
6,7684,1619,6,74,0,1205,79,92,696,208,...,0,0,0,0,0,50,0,174,0,0
7,13038,1917,0,108,991,134,681,171,524,0,...,255,0,144,0,0,0,0,11,46,0
8,640,1,0,1,128,168,2601,2421,1,0,...,1,0,187,0,51,14,0,86,0,0
9,3,1,0,1794,256,1075,3335,4781,1,4729,...,71,0,1,0,0,0,0,61,0,0
10,542,81,0,99,180,183,660,93,77,0,...,47,49,4,26,0,151,0,15,31,0


In [9]:
log_reg = LogisticRegression(penalty="l1", C=1.2, solver="liblinear", random_state=42)
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

1.0

In [12]:
y_pred = log_reg.predict(X_pred)
y_pred

array([1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1], dtype=int64)

In [13]:
np.sum(y_pred == 1)

24

In [14]:
from lightgbm import LGBMClassifier

In [18]:
clf = LGBMClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [19]:
cross_val_score(clf, X_train, y_train, cv=10)

array([1.        , 0.93333333, 1.        , 0.93333333, 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.92857143])

In [ ]:
with open("predictions/smartseq_hcc1806_predictions.txt", 'w') as file:
        for n in y_pred:
            label = "Normoxia" if n == 1 else "Hypoxia"
            file.write(label + '\n')

In [11]:
cross_val_score(xgb.XGBClassifier(alpha=1e-05), X, y).mean()

0.9777777777777779

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self, D, H, H_2, H_3):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Linear(D, H),
            nn.ReLU(),
            nn.Linear(H, H_2),
            nn.ReLU(),
            nn.Linear(H_2, H_3),
            nn.ReLU(),
            nn.Linear(H_3, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.stack(x)
    

def train_loop(X, y, eta, epochs, H, H_2, H_3, weight_decay, epoch_print):
    N, D = X.shape
    
    model = NeuralNetwork(D, H, H_2, H_3)
    optimizer = torch.optim.AdamW(model.parameters(), lr=eta, weight_decay=weight_decay)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        y_pred = model(X)
        loss = criterion(y_pred, y)
        if epoch % epoch_print == 0:
            print(epoch, loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return model

In [41]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("label", axis=1), df["label"], test_size=0.2)
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = y_test.tolist()

In [42]:
# X_train, X_test, y_train, y_test = train_test_split(df.drop("label", axis=1), df["label"], test_size=0.2)
# X_train = torch.tensor(X_train.values, dtype=torch.float32)
# y_train = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32)
# X_test = torch.tensor(X_test.values, dtype=torch.float32)
# y_test = y_test.tolist()


model = train_loop(X_train, y_train, 0.001, 1001, 16, 16, 16, 0, 1000)

with torch.no_grad():
    y_pred = [1 if x > 0.5 else 0 for x in model(X_test).view(-1).tolist()]


right = 0
N = len(y_test)
for i in range(N):
    if y_pred[i] == y_test[i]:
        right += 1
accuracy = right / N
print("Accuracy:", accuracy)



0 11.709053039550781
1000 1.379310965538025
Accuracy: 0.972972972972973


In [39]:
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(df.drop("label", axis=1), df["label"], test_size=0.2)

# Initialize the XGBoost classifier
clf = xgb.XGBClassifier()

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8918918918918919
Precision: 0.8888888888888888
Recall: 0.8888888888888888
F1 Score: 0.8888888888888888
